In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 9.1 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=5da1e978d09ce575dc883c830f904684849413b2856c6f700eee8c849f05a6f0
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [2]:
! signate list
! signate files --competition-id=492
! signate download --competition-id=492

  competitionId  title                                                                  closing     prize            submitters
---------------  ---------------------------------------------------------------------  ----------  -------------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                   -                                  5172
             24  【練習問題】お弁当の需要予測                                           -                                  6343
             27  【練習問題】Jリーグの観客動員数予測                                    -                                  1433
            100  【練習問題】手書き文字認識                                             -           Knowledge               174
            102  【練習問題】タイタニックの生存予測                                     -           Knowledge              1064
            103  【練習問題】音楽ラベリング                                             -           Knowledge                61
            104  【練習問題】スパムメール分類                                     

In [3]:
# ディレクトリ内のデータの確認
%ls

adc.json                                       requirements.txt
category_names.csv                             sales_history.csv
item_categories.csv                            sample_data/
PBL01_h_exercise4_question.pdf                 sample_submission.csv
PBL01_i_exercise4_supplementary-documents.pdf  test.csv
PBL01_sample_code.ipynb


In [4]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#baseline

In [5]:
!pip install pycaret

     |████████████████████████████████| 266 kB 5.0 MB/s 
     |████████████████████████████████| 111 kB 53.5 MB/s 
     |████████████████████████████████| 6.8 MB 43.5 MB/s 
     |████████████████████████████████| 271 kB 64.6 MB/s 
     |████████████████████████████████| 14.6 MB 121 kB/s 
     |████████████████████████████████| 1.7 MB 57.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 68.6 MB/s 
     |████████████████████████████████| 2.0 MB 42.4 MB/s 
     |████████████████████████████████| 261 kB 74.3 MB/s 
     |████████████████████████████████| 167 kB 71.6 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 80 kB 10.1 MB/s 
     |████████████████████████████████| 62 kB 837 kB/s 
     |████████████████████████████████| 636 kB 54.6 MB/s 
     |███████████████████

In [ ]:
'''
初期設定
'''

DATA_PATH = '/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/processed_train_test_df.csv'
SAMPLESUB_PATH = 'sample_submission.csv'
SUB_PATH = '/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/submit/demand_RandomForest.csv'


# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from pycaret.regression import *

# データの読み込み
df = pd.read_csv(DATA_PATH)
print(df.isnull().sum())
print(df.dtypes)

# object型の変数の取得
categories = df.columns[df.dtypes == 'object']
print(categories)

# label Encoding
for cat in categories:
    le = LabelEncoder() 
    print(cat)
    
    df[cat].fillna('missing', inplace=True)
    le = le.fit(df[cat])
    df[cat] = le.transform(df[cat])
    # LabelEncoderは数値に変換するだけであるため、最後にastype('category')としておく
    df[cat] = df[cat].astype('category') 

# trainとtestに分割
train = df[~df['sales'].isnull()]
test = df[df['sales'].isnull()]
print(train.head())
exp1 = setup(train, target = 'sales')

# モデルの比較
compare_models()

,Description,Value
0,session_id,826
1,Target,sales
2,Original Data,"(1648926, 22)"
3,Missing Values,False
4,Numeric Features,17
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1154248, 59)"


IntProgress(value=0, description='Processing: ', max=89)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:14:39
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.7281,4.875100e+00,2.2040,4.633000e-01,0.4667,0.7098,274.129
dt,Decision Tree Regressor,0.7836,5.402700e+00,2.3200,4.047000e-01,0.5400,0.8598,20.618
ridge,Ridge Regression,0.9083,7.834800e+00,2.7929,1.400000e-01,0.5326,0.5894,0.877
br,Bayesian Ridge,0.9083,7.834900e+00,2.7929,1.400000e-01,0.5326,0.5892,8.416
lr,Linear Regression,0.9198,7.866700e+00,2.7987,1.365000e-01,0.5388,0.6054,2.981
omp,Orthogonal Matching Pursuit,0.9308,7.929100e+00,2.8098,1.295000e-01,0.5354,0.5918,0.611
en,Elastic Net,0.9577,8.159700e+00,2.8508,1.038000e-01,0.5519,0.4862,0.844
huber,Huber Regressor,0.7725,8.291300e+00,2.8735,8.950000e-02,0.5101,0.7429,68.578
lasso,Lasso Regression,0.9867,8.330200e+00,2.8806,8.480000e-02,0.5650,0.4633,0.736
llar,Lasso Least Angle Regression,1.0785,9.092200e+00,3.0103,-0.000000e+00,0.6186,0.4401,0.613
